In [4]:
import requests
from pathlib import Path

# === CONFIG ===
FASTAPI_URL = "http://fastapi_server:8000/predict"
AUDIO_PATH = "Sample_Audio/iNat51530.ogg"  # Or "sample.wav"
NUM_REQUESTS = 100

# === Send file 100 times ===
for i in range(NUM_REQUESTS):
    with open(AUDIO_PATH, "rb") as f:
        files = [("files", (Path(AUDIO_PATH).name, f, "audio/ogg"))]  # Use "audio/wav" for .wav files
        response = requests.post(FASTAPI_URL, files=files)
        print(f"[{i+1}] Status: {response.status_code}, Response: {response.json()}")

[1] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[2] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[3] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[4] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[5] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[6] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[7] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[8] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no attribute 'inc'"}]
[9] Status: 200, Response: [{'filename': 'iNat51530.ogg', 'error': "'Histogram' object has no at

In [5]:
import time
import requests
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

FASTAPI_URL = "http://fastapi_server:8000/predict"
AUDIO_PATH = "Sample_Audio/iNat51530.ogg"
load_pattern = [10, 20, 30, 50, 30, 20, 1]
delay_between_steps = 30  # seconds

audio_bytes = Path(AUDIO_PATH).read_bytes()

def send_continuous_requests(duration_sec):
    start_time = time.time()
    while time.time() - start_time < duration_sec:
        try:
            files = {"files": (Path(AUDIO_PATH).name, audio_bytes, "audio/ogg")}
            response = requests.post(FASTAPI_URL, files=files, timeout=10)
            print(f"[{time.strftime('%X')}] Status: {response.status_code}")
        except Exception as e:
            print(f"[{time.strftime('%X')}] Error: {e}")
        time.sleep(1)

def run_load_stage(concurrent_workers, duration_sec):
    print(f"\n==> Starting stage: {concurrent_workers} workers for {duration_sec}s")
    with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = [executor.submit(send_continuous_requests, duration_sec) for _ in range(concurrent_workers)]
        for f in futures:
            f.result()

for load in load_pattern:
    run_load_stage(load, delay_between_steps)



==> Starting stage: 10 workers for 30s
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:00] Status: 200
[03:21:01] Status: 200
[03:21:02] Status: 200
[03:21:02] Status: 200
[03:21:02] Status: 200
[03:21:02] Status: 200
[03:21:02] Status: 200
[03:21:02] Status: 200
[03:21:02] Status: 200
[03:21:02] Status: 200
[03:21:03] Status: 200
[03:21:03] Status: 200
[03:21:03] Status: 200
[03:21:03] Status: 200
[03:21:03] Status: 200
[03:21:03] Status: 200
[03:21:03] Status: 200
[03:21:03] Status: 200
[03:21:04] Status: 200
[03:21:04] Status: 200
[03:21:04] Status: 200
[03:21:04] Status: 200
[03:21:04] Status: 200
[03:21:04] Status: 200
[03:21:04] Status: 200
[03:21:04] Status: 200
[03:21:05] Status: 200
[03:21:05] Status: 200
[03:21:05] Status: 200
[03:21:05] Status: 200
[03:21:05] Status: 200
[03:21:05] Status: 200
[03:21:05] Status

In [1]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor

# === Configuration ===
FASTAPI_URL = "http://fastapi_server:8000/predict"
AUDIO_FILE = "Sample_Audio/iNat51530.ogg"  # Must exist in the same directory
CONTENT_TYPE = "audio/ogg"

load_pattern = [1, 2, 3, 5, 3, 2, 1]  # number of concurrent users per stage
delay_between_steps = 60  # how long each stage runs (in seconds)

# === Send continuous requests for a given duration ===
def send_continuous_requests(duration_sec):
    start_time = time.time()
    while time.time() - start_time < duration_sec:
        try:
            with open(AUDIO_FILE, "rb") as f:
                files = [("files", (AUDIO_FILE, f, CONTENT_TYPE))]
                response = requests.post(FASTAPI_URL, files=files, timeout=10)
                print(f"[{response.status_code}] {response.elapsed.total_seconds():.2f}s")
        except Exception as e:
            print(f"[ERROR] {e}")
        time.sleep(0.5)  # short delay between calls per user

# === Run load stage with N concurrent users ===
def run_load_stage(concurrent_users, duration_sec):
    print(f"\nLoad Stage: {concurrent_users} users for {duration_sec}s")
    with ThreadPoolExecutor(max_workers=concurrent_users) as executor:
        futures = [executor.submit(send_continuous_requests, duration_sec) for _ in range(concurrent_users)]
        for f in futures:
            f.result()

# === Execute each load pattern stage ===
if __name__ == "__main__":
    for load in load_pattern:
        run_load_stage(load, delay_between_steps)



Load Stage: 1 users for 60s
[200] 1.12s
[200] 0.53s
[200] 0.54s
[200] 0.52s
[200] 0.20s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.21s
[200] 0.18s
[200] 0.20s
[200] 0.18s
[200] 0.18s
[200] 0.19s
[200] 0.20s
[200] 0.16s
[200] 0.18s
[200] 0.17s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.17s
[200] 0.20s
[200] 0.18s
[200] 0.17s
[200] 0.19s
[200] 0.20s
[200] 0.18s
[200] 0.18s
[200] 0.21s
[200] 0.19s
[200] 0.17s
[200] 0.17s
[200] 0.17s
[200] 0.19s
[200] 0.19s
[200] 0.18s
[200] 0.20s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.20s
[200] 0.18s
[200] 0.17s
[200] 0.18s
[200] 0.17s
[200] 0.19s
[200] 0.17s
[200] 0.18s
[200] 0.20s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.17s
[200] 0.21s
[200] 0.19s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.17s
[200] 0.19s
[200] 0.17s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.18s
[200] 0.19s